In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skvideo.io
import csv
import os
from tqdm import tqdm
import cv2 as cv
from scipy.optimize import minimize
%matplotlib inline
# np.set_printoptions(threshold=sys.maxsize)

In [ ]:
asset_path = '../train/assets'
model_name = 'keras'
batch_size = 2
num_epoch = 20
steps_per_epoch = 400
train_name = f'model={model_name}-batch_size={batch_size}-num_epoch={train_iters}-steps_per_epoch={steps_per_epoch}'
history_loc = os.path.join(asset_path, )

In [ ]:
# Global Vars
#img_pix_count = (cv.imread('../train/photos/images/gray_Clipped/0.jpg'))[100:440, :-90].shape
img_pix_count = (cv.imread('../train/photos/images/gray_Clipped/0.jpg')).shape
img_pix_count = (img_pix_count[0] * img_pix_count[1])*3
print('Pixel Count:', img_pix_count)

In [ ]:
def adjust_Value(prvs_FRAME, curr_FRAME, HSV_value):
    prvs_FRAME = cv.cvtColor(prvs_FRAME, cv.COLOR_BGR2HSV)
    curr_FRAME = cv.cvtColor(curr_FRAME, cv.COLOR_BGR2HSV)
    prvs_FRAME[...,2] = prvs_FRAME[...,2]*HSV_value
    curr_FRAME[...,2] = curr_FRAME[...,2]*HSV_value
    return prvs_FRAME, curr_FRAME

In [ ]:
def opticalFlow_DENSE(prvs_FRAME, curr_FRAME):
    # flow_mat = None
    # image_scale = 0.5
    # pyr_images = 1 # was 3
    # win_size = 15
    # pyr_iterations = 2 # was 3
    # poly_expans = 5
    # std = 1.3 # was 1.2
    
    flow_mat = None
    image_scale = 0.5
    pyr_images = 1 # was 3
    win_size = 10
    pyr_iterations = 2 # was 3
    poly_expans = 5
    std = 1.3 # was 1.2

    hsv = np.zeros_like(prvs_FRAME)
    hsv[...,1] = 255
    prvs_FRAME = cv.cvtColor(prvs_FRAME,cv.COLOR_BGR2GRAY)
    curr_FRAME = cv.cvtColor(curr_FRAME,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs_FRAME,curr_FRAME,flow_mat,image_scale,pyr_images,win_size,pyr_iterations,poly_expans,std,0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    return bgr

In [ ]:
def getData(train_meta, kLoopCount, learn_index):
    #print('In getData')
    
    global img_pix_count
    opt_flows = np.empty((0,img_pix_count), dtype='uint8')
    speed = np.empty((0))
    for k in range(kLoopCount):
        HSV_value = np.random.uniform(low=.7, high=1.4)
        curr_FRAME = cv.imread(train_meta['image_path'][learn_index[k]])
        #curr_FRAME = curr_FRAME[100:440, :-90]
        prvs_FRAME = cv.imread(train_meta['image_path'][learn_index[k]-1])
        #prvs_FRAME = prvs_FRAME[100:440, :-90]
        speed = np.append(speed, train_meta['speed'][learn_index[k]])
        prvs_FRAME, curr_FRAME = adjust_Value(prvs_FRAME, curr_FRAME, HSV_value)
        opt_flow = opticalFlow_DENSE(prvs_FRAME, curr_FRAME)
        opt_flow.shape = img_pix_count
        opt_flow.dtype = 'uint8'
        opt_flows = np.append(opt_flows, [opt_flow], axis=0)
    #print('Opt FLOWS Shape:', opt_flows.shape, 'Type:', opt_flows.dtype)
    #print('Speed Shape:', speed.shape, 'Type:', speed.dtype)
    
    #print('Out getData')
    #print()
    return opt_flows, speed

In [ ]:
def getTrainingData(train_meta, train_index, kLoopCount):
    #print('In startLearn')
    
    train_frames = train_meta.shape[0]
    train_length = int(.8*(train_frames-1))
    #print('Train Length:', train_length)
    data_index = np.loadtxt('../train/text/train_val_index.txt', np.dtype('int16'), delimiter='\n')
    learn_index = data_index[0:train_length]
    
    
    opt_flow, speed = getData(train_meta, kLoopCount, learn_index[index:index+kLoopCount])
    
    return opt_flow, speed

In [ ]:
def getValData(train_meta, kLoopCount, learnLoopCount):
    #print('In startValCheck')
    
    train_frames = train_meta.shape[0]
    train_length = int(.8*train_frames-1)
    val_length = int((train_frames-1) - train_length)
    #val_length = int(.2*train_frames-1)
    #print('Train Length:', train_length)
    data_index = np.loadtxt('../train/text/train_val_index.txt', np.dtype('int16'), delimiter='\n')
    val_index = data_index[train_length:]
    #print('Val Index:', learn_index.shape)
    
    #iLoopCount = 1
    #kLoopCount = 1
    jLoopCount = int(val_length/kLoopCount) if val_length%kLoopCount==0 else (int(val_length/kLoopCount)+1)
    #learnLoopCount = 10
    #m = kLoopCount
    
    getCost(train_meta, val_index, val_length, jLoopCount, kLoopCount)

In [ ]:
def createCNNModel():
    model = Sequential()
    model.add(Conv2D(40, (5,5), (3,3), activation='relu', input_shape=(320,136,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(20, (5,5), (2,2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
def start(iLoopCount, kLoopCount, learnLoopCount, costOnLoop, alpha_arg, lmbd_arg):
    train_csv_path = '../train/text/CSV'
    train_meta = pd.read_csv(os.path.join(train_csv_path, 'trainGrayClipped_meta.csv'))
    
    data_index = np.loadtxt('../train/text/train_val_index.txt', np.dtype('int16'), delimiter='\n')
    dataset_frames_num = train_meta.shape[0]
    train_frames_num = int(.8*(train_frames-1))
    
    getTrainingData(train_meta, train_index, kLoopCount)
    plotCost()
    getValData(train_meta, kLoopCount, learnLoopCount)
    
    model = createCNNModel
    

In [ ]:
start(20, 2, 1, 19, alpha, lmbd)